In [755]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
# from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot, Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,Input,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Dense,AdditiveAttention,MultiHeadAttention,Bidirectional,LSTM,Flatten,Cropping3D,UpSampling3D,Attention,Dropout
import tensorflow.compat.v1 as tf

In [756]:
# !pip install keras_preprocessing

In [757]:
# importing data
data_train=pd.read_csv('/kaggle/input/twitter-data-set/twitter_training.csv')
data_test=pd.read_csv('/kaggle/input/twitter-data-set/twitter_validation.csv')

In [758]:
data_train.shape,data_test.shape

((74681, 4), (999, 4))

In [759]:
data_train.columns=['no1','no2','sentiment','text']

In [760]:
data_train.drop('no1',axis=1,inplace=True)
data_train.drop('no2',axis=1,inplace=True)
data_train.head()

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [761]:
data_test.columns=['no1','no2','sentiment','text']

In [762]:
data_test.drop('no1',axis=1,inplace=True)
data_test.drop('no2',axis=1,inplace=True)
data_test

,sentiment,text
0,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,Negative,@Microsoft Why do I pay for WORD when it funct...
2,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,Neutral,Now the President is slapping Americans in the...
4,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...
994,Irrelevant,⭐️ Toronto is the arts and culture capital of ...
995,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
996,Positive,Today sucked so it’s time to drink wine n play...
997,Positive,Bought a fraction of Microsoft today. Small wins.


In [763]:
data_train.dropna(inplace=True)
data_test.dropna(inplace=True)

In [764]:
data_train.shape,data_test.shape

((73995, 2), (999, 2))

In [765]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [766]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [767]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    
    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [768]:
x_train_text = []
sentences = list(data_train['text'])
for sen in sentences:
    x_train_text.append(preprocess_text(sen))
    
    
x_test_text = []
sentences = list(data_test['text'])
for sen in sentences:
    x_test_text.append(preprocess_text(sen))

In [769]:
data_train['sentiment']=data_train['sentiment'].map({'Positive':0,'Negative':2,'Neutral':1,'Irrelevant':3})
data_test['sentiment']=data_test['sentiment'].map({'Positive':0,'Negative':2,'Neutral':1,'Irrelevant':3})

In [770]:
data_train['sentiment'].head(30)


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    1
12    1
13    1
14    1
15    1
16    1
17    0
18    0
19    0
20    0
21    0
22    0
23    2
24    2
25    2
26    2
27    2
28    2
29    0
Name: sentiment, dtype: int64

In [771]:
x_train_sentiment=np.array(data_train.iloc[:,0])
x_train_sentiment=tf.one_hot(x_train_sentiment, 4)
x_train_sentiment

<tf.Tensor: shape=(73995, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>

In [772]:
x_test_sentiment=np.array(data_test.iloc[:,0])
x_test_sentiment=tf.one_hot(x_test_sentiment, 4)
x_test_sentiment

<tf.Tensor: shape=(999, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)>

In [773]:
x_train=x_train_text
y_train=x_train_sentiment
x_test=x_test_text
y_test=x_test_sentiment

In [774]:
y_train.shape,y_test.shape

(TensorShape([73995, 4]), TensorShape([999, 4]))

In [775]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(x_train)

x_train = word_tokenizer.texts_to_sequences(x_train)
x_test = word_tokenizer.texts_to_sequences(x_test)


In [776]:
vocab_length = len(word_tokenizer.word_index) +1

vocab_length


29131

In [777]:
maxlen = 100

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [778]:
x_train.shape,y_train.shape

((73995, 100), TensorShape([73995, 4]))

In [779]:
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/kaggle/input/kaggleinputglovedataglove6b200dtxt/a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [780]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [781]:
embedding_matrix.shape
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.94352001,  0.18712001,  0.36660001, ..., -0.11644   ,
        -0.47114   , -0.40741   ],
       [ 0.062659  ,  0.026003  ,  1.12199998, ...,  0.46125999,
         0.88780999, -0.031292  ],
       ...,
       [-0.29304001, -0.12909   , -0.0089504 , ..., -0.57717001,
         0.29337001,  0.16899   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.1246    , -0.21162   , -0.36636001, ..., -0.093049  ,
         0.24579   ,  0.36520001]])

In [782]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [783]:
# model=Sequential()

In [784]:
# model.add(Embedding(vocab_length,100,weights=[embedding_matrix], input_length=maxlen))
# model.add(Conv1D(64,3,padding ="same",activation="relu"))
# model.add(MaxPooling1D())
# model.add(Dropout(0.2))
# model.add(Conv1D(128,5,padding="same",activation="relu"))
# #model.add(MultiHeadAttention(key_dim=128, value_dim=128, num_heads=1,value=10))
# model.add(MaxPooling1D())
# model.add(Dropout(0.2))
# model.add(Conv1D(128,7,padding="same",activation="relu"))
# #model.add(Bidirectional(LSTM(20)))
# model.add(Bidirectional(LSTM(64, return_sequences=True)))
# model.add(Bidirectional(LSTM(128)))

# # model.add(attention())
# #model.add(Flatten())
# # model.add(Cropping3D(cropping=((1, 1), (1, 1), (1, 1))))
# # model.add(UpSampling3D(size=(2, 2, 2)))
# # model.add(AdditiveAttention()) 
# #model.add(MultiHeadAttention(num_heads=2, key_dim=128, value=128))


# model.add(Dense(units=4,activation="sigmoid"))

In [785]:
embedding_input = Input(shape=(maxlen,))
embedding_output=Embedding(vocab_length,100,weights=[embedding_matrix], input_length=maxlen)(embedding_input)
CNN_output=Conv1D(128,3,padding ="same",activation="relu")(embedding_output)
# maxpool_output=MaxPooling1D()(CNN_output)
# dropout_output=Dropout(0.1)(CNN_output)
CNN_output=Conv1D(256,5,padding="same",activation="relu")(CNN_output)
maxpool_output=MaxPooling1D()(CNN_output)
# dropout_output=Dropout(0.1)(maxpool_output)
CNN_output=Conv1D(512,7,padding="same",activation="relu")(maxpool_output)
BiLSTM_output=Bidirectional(LSTM(128, return_sequences=True))(CNN_output)
attention_output = Attention()([BiLSTM_output, BiLSTM_output])
BiLSTM_output=Bidirectional(LSTM(256))(attention_output)

l=Dense(units=4,activation="sigmoid")(BiLSTM_output)

In [786]:
model = tf.keras.Model(inputs=[embedding_input], outputs = [l])

In [787]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [788]:
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_25 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 embedding_24 (Embedding)       (None, 100, 100)     2913100     ['input_25[0][0]']               
                                                                                                  
 conv1d_55 (Conv1D)             (None, 100, 128)     38528       ['embedding_24[0][0]']           
                                                                                                  
 conv1d_56 (Conv1D)             (None, 100, 256)     164096      ['conv1d_55[0][0]']              
                                                                                           

In [789]:
model.fit(x_train,y_train,epochs=16,batch_size=32,validation_data=(x_test,y_test))

Epoch 1/16
2313/2313 [==============================] - 94s 37ms/step - loss: 0.3961 - accuracy: 0.6138 - val_loss: 0.2048 - val_accuracy: 0.8458
Epoch 2/16
2313/2313 [==============================] - 54s 23ms/step - loss: 0.1834 - accuracy: 0.8429 - val_loss: 0.0977 - val_accuracy: 0.9389
Epoch 3/16
2313/2313 [==============================] - 55s 24ms/step - loss: 0.1029 - accuracy: 0.9116 - val_loss: 0.0800 - val_accuracy: 0.9570
Epoch 4/16
2313/2313 [==============================] - 53s 23ms/step - loss: 0.0724 - accuracy: 0.9368 - val_loss: 0.0741 - val_accuracy: 0.9580
Epoch 5/16
2313/2313 [==============================] - 52s 23ms/step - loss: 0.0572 - accuracy: 0.9492 - val_loss: 0.1022 - val_accuracy: 0.9449
Epoch 6/16
2313/2313 [==============================] - 53s 23ms/step - loss: 0.0495 - accuracy: 0.9546 - val_loss: 0.0814 - val_accuracy: 0.9590
Epoch 7/16
2313/2313 [==============================] - 52s 23ms/step - loss: 0.0429 - accuracy: 0.9598 - val_loss: 0.0880 -

In [790]:
y_test_pred=model.predict(x_test)
model.evaluate(x_test,y_test) 

32/32 [==============================] - 0s 9ms/step - loss: 0.0993 - accuracy: 0.9620


[0.09927903860807419, 0.9619619846343994]

In [791]:
y_test_pred

array([[9.1668085e-08, 9.9999893e-01, 6.5697418e-07, 1.2335552e-06],
       [3.6128458e-05, 6.8111274e-07, 9.9996448e-01, 1.0204661e-05],
       [6.9168455e-05, 2.0818125e-06, 9.9990273e-01, 1.7828315e-05],
       ...,
       [9.9996305e-01, 1.1147964e-05, 5.3596768e-06, 1.6973943e-05],
       [9.9998927e-01, 1.7192616e-06, 2.2223610e-06, 6.0489010e-06],
       [1.1905720e-07, 9.9999869e-01, 9.1129033e-07, 1.4751417e-06]],
      dtype=float32)

In [792]:
input_array=y_test_pred
binary_array = np.zeros_like(input_array)
max_indices = np.argmax(input_array, axis=1)
binary_array[np.arange(len(input_array)), max_indices] = 1
y_test_pred=binary_array
print(binary_array)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


In [793]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       277
           1       0.97      0.94      0.96       285
           2       0.96      0.98      0.97       266
           3       0.93      0.98      0.96       171

   micro avg       0.96      0.96      0.96       999
   macro avg       0.96      0.96      0.96       999
weighted avg       0.96      0.96      0.96       999
 samples avg       0.96      0.96      0.96       999

